# Query Data using LLM

Here is the overall RAG pipeline.   In this notebook, we will do steps (6), (7), (8), (9) and (10)
- Importing data is already done in this notebook [rag_2_load_data_into_milvus.ipynb](rag_2_load_data_into_milvus.ipynb)
- 👉 Step 6: Calculate embedding for user query
- 👉 Step 7 & 8: Send the query to vector db to retrieve relevant documents
- 👉 Step 9 & 10: Send the query and relevant documents (returned above step) to LLM and get answers to our query

![image missing](media/rag-overview-2.png)

## Step-1: Configuration

In [1]:
from my_config import MY_CONFIG

## Step-2: Connect to Vector Database

Milvus can be embedded and easy to use.

<span style="color:blue;">Note: If you encounter an error about unable to load database, try this: </span>

- <span style="color:blue;">In **vscode** : **restart the kernel** of previous notebook. This will release the db.lock </span>
- <span style="color:blue;">In **Jupyter**: Do `File --> Close and Shutdown Notebook` of previous notebook. This will release the db.lock</span>
- <span style="color:blue;">Re-run this cell again</span>


In [2]:
# connect to vector db
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.milvus import MilvusVectorStore

vector_store = MilvusVectorStore(
    uri = MY_CONFIG.DB_URI ,
    dim = MY_CONFIG.EMBEDDING_LENGTH , 
    collection_name = MY_CONFIG.COLLECTION_NAME,
    overwrite=False  # so we load the index from db
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

print ("✅ Connected to Milvus instance: ", MY_CONFIG.DB_URI )

/home/sujee/my-stuff/ai-alliance/data-prep-kit/data-prep-kit-examples-dev-2/.venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'validate_default' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'validate_default' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/home/sujee/my-stuff/ai-alliance/data-prep-kit/data-prep-kit-examples-dev-2/.venv/lib/python3.12/site-packages/milvus_lite/__init__.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this pa

✅ Connected to Milvus instance:  ./rag_1_dpk.db


## Step-3: Setup Embeddings

Use the same embeddings we used to index our documents!

In [3]:
# If connection to https://huggingface.co/ failed, uncomment the following path
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

In [4]:

from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

print ("✅ Using embedding Model:", MY_CONFIG.EMBEDDING_MODEL)
print ("✅ Using embedding length:", MY_CONFIG.EMBEDDING_LENGTH)

Settings.embed_model = HuggingFaceEmbedding(
    model_name = MY_CONFIG.EMBEDDING_MODEL
)

/home/sujee/my-stuff/ai-alliance/data-prep-kit/data-prep-kit-examples-dev-2/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Using embedding Model: sentence-transformers/all-MiniLM-L6-v2
✅ Using embedding length: 384


## Step-4: Load Document Index from DB

In [5]:
%%time

from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, storage_context=storage_context)

print ("✅ Loaded index from vector db:", MY_CONFIG.DB_URI )

✅ Loaded index from vector db: ./rag_1_dpk.db
CPU times: user 92.2 ms, sys: 17.3 ms, total: 110 ms
Wall time: 109 ms


## Step-6: Using LLM

We can use LLMs running on remote services or locally (e.g. using Ollama).  We use [LiteLLM library](https://docs.litellm.ai/docs/) to choose LLM runtime.

Here are some examples.

- [Nebuis Token Factory](https://tokenfactory.nebius.com/)
- [replicate.com](https://replicate.com)

**How to use the LLM inference services**

If Using Nebius

Update `.env` file as follows

```ini
LLM_MODEL = 'nebius/openai/gpt-oss-120b'
NEBIUS_API_KEY = 'your key goes here'
```

If using Replicate

```ini
LLM_MODEL = 'ibm-granite/granite-3.3-8b-instruct'
REPLICATE_API_TOKEN=xyz
```


In [6]:
from llama_index.llms.litellm import LiteLLM

# Setup LLM
print (f"✅ Using LLM model : {MY_CONFIG.LLM_MODEL}")
Settings.llm = LiteLLM (
        model=MY_CONFIG.LLM_MODEL,
    )

✅ Using LLM model : nebius/openai/gpt-oss-120b


## Step-7: Query

In [7]:
%%time 

import query_utils

question = "How were Granite models trained?"
query_engine = index.as_query_engine()
query = query_utils.tweak_query(question, MY_CONFIG.LLM_MODEL)
res = query_engine.query(query)
print(res)

The Granite models were built using a multi‑stage process. First, a comprehensive data‑collection pipeline gathered raw material, which was then filtered and pre‑processed to create a clean training set. The resulting dataset fed into the model architecture described in the architecture section. The actual training—covering model size, optimization settings, compute resources, and training duration—is detailed in the training section of the paper. After the base model was trained, an instruction‑tuning phase refined its behavior for code‑related tasks. The full experimental evaluation of the trained models appears later in the results section.
CPU times: user 192 ms, sys: 138 ms, total: 330 ms
Wall time: 2.84 s


In [8]:
%%time 

import query_utils

question = "What is attention mechanism?"
query_engine = index.as_query_engine()
query = query_utils.tweak_query(question, MY_CONFIG.LLM_MODEL)
res = query_engine.query(query)
print(res)

The attention mechanism is a way for a neural model to dynamically weight the relevance of different tokens when processing a sequence. It works by projecting each token into three vectors—queries, keys, and values—and then computing similarity scores between queries and keys. These scores are turned into a probability distribution (via soft‑max) that determines how much each value contributes to the final representation of a token.  

In a Transformer, this process is performed in parallel across multiple “heads,” each learning to focus on different kinds of relationships (e.g., long‑distance dependencies, anaphora, syntactic structure). The mechanism is used in three main configurations:

1. **Encoder‑decoder attention** – queries come from the decoder while keys and values come from the encoder, letting every decoder position attend to the entire encoded input.  
2. **Encoder self‑attention** – queries, keys, and values all originate from the encoder’s previous layer, enabling each 

In [9]:
%%time 

import query_utils

question = "When was the moon landing?"
query_engine = index.as_query_engine()
query = query_utils.tweak_query(question, MY_CONFIG.LLM_MODEL)
res = query_engine.query(query)
print(res)

I’m sorry, but I don’t have that information.
CPU times: user 43.7 ms, sys: 4.86 ms, total: 48.5 ms
Wall time: 1.02 s
